In [1]:
import tensorflow as tf

In [29]:
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Dropout
from tensorflow.keras import Model

In [3]:
# 加载并准备 MNIST 数据集。
mnist = tf.keras.datasets.mnist

In [8]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [23]:
type(x_train)
x_train.shape

(60000, 28, 28, 1)

In [26]:
np.unique(y_train)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

In [13]:
# 在当前数据上 新增维度 1 reshape()
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

In [21]:
import numpy as np

In [24]:
x_train.shape

(60000, 28, 28, 1)

In [27]:
#使用 tf.data 来将数据集切分为 batch 以及混淆数据集：
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [30]:
#使用 Keras 模型子类化（model subclassing） API 构建 tf.keras 模型：
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    # 32:输出单元数,  3:卷积核大小, activition: 激活函数
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10, activation='softmax')
    self.d3 = Dropout(0.2) # dropout 防止过拟合
    

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    x = self.d3(x)
    return self.d2(x)

model = MyModel()

In [31]:
# 损失函数交叉熵
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

In [32]:
# 训练损失
train_loss = tf.keras.metrics.Mean(name='train_loss')
# 训练准确度
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
# 测试损失
test_loss = tf.keras.metrics.Mean(name='test_loss')
# 测试精确度
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [33]:
# 使用 tf.GradientTape 来训练模型：
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

In [34]:
#测试模型：
@tf.function
def test_step(images, labels):
  predictions = model(images)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [35]:
EPOCHS = 5

for epoch in range(EPOCHS):
  # 在下一个epoch开始时，重置评估指标
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print (template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         test_loss.result(),
                         test_accuracy.result()*100))


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1, Loss: 0.13519197702407837, Accuracy: 95.94166564941406, Test Loss: 0.07092384248971939, Test Accuracy: 97.65999603271484
Epoch 2, Loss: 0.0409700945019722, Accuracy: 98.73666381835938, Test Loss: 0.055926866829395294, Test Accuracy: 98.07999420166016
Epoch 3, Loss: 0.021301837638020515, Accuracy: 99.34333801269531, Test Loss: 0.05419931188225746, Test Accuracy: 98.29999542236328
Epoch 4, Loss: 0.01314876601099968, Accuracy: 99.57167053222656, Test Loss: 0.059581246227025986, Test Accuracy: 98.3